STEP 1: UPLOAD DATASET TO COLAB

In [1]:
#upload the dataset into google colab
from google.colab import files
uploaded = files.upload()

Saving tbcaseskibongoto.csv to tbcaseskibongoto.csv


IMPORT NECESSARY LIBRARY

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(10,5), 'figure.dpi':100})
import datetime



LOADING THE DATASET

In [ ]:
pf = pd.read_csv('tbcaseskibongoto.csv')

In [ ]:
pf=pd.DataFrame(pf)

In [ ]:
pf

In [ ]:
pf.dropna()

In [ ]:
pf.info()

In [ ]:
pf['date']=pd.to_datetime(pf['date'])

In [ ]:
pf.info()

In [ ]:
pf=pf[['date','tbcases']]

In [ ]:
pf

In [ ]:
pf=pf.set_index('date')

VISUALIZING THE DATASET

In [ ]:
#pf.plot(x='Month',title="Kibongoto National TB Hospital TB cases since January 2015- December s2020",ylabel="TB cases")
#plt.show()
pf.plot(figsize = (15, 6),title="Monthly New and Relapse (bacteriologically or clinical diagnosis) TB incidence at Kibongoto Infectious Disease Hospital  from 2015 to 2020.",ylabel="New and Relapse TB cases",xlabel="Year")
plt.show()

DECOMPOSE TIME SERIES DATA

In [ ]:
decomposition = sm.tsa.seasonal_decompose(pf, model='additive',freq=12,extrapolate_trend='freq')
plt.rcParams["figure.figsize"] = [16,9]
fig = decomposition.plot()

PlOT ACF AND PACF

In [ ]:
import statsmodels.graphics.tsaplots as sgt
sgt.plot_acf(pf)
plt.show()

In [ ]:
sgt.plot_pacf(pf)
plt.show()

CHECK FOR THE STATIONARY

In [ ]:
pf=pf.squeeze()

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller_results=adfuller(pf,autolag='AIC')
print(f'adfuller statistics:{adfuller_results[0]}')
print(f'p-value: {adfuller_results[1]}')
for key,value in adfuller_results[4].items():
  print("Critical value: ")
  print(f'{key},{value}')

SPLITTING INTO TRAINING AND TESTING SET

In [ ]:
# split into train and test sets
X = pf.values
size = int(len(X) * 0.7)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

In [ ]:
type(pf)

In [ ]:
import itertools 

In [ ]:
p = range(0, 3)
d = range(0,1)
q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))



In [ ]:
metric_aic_dict=dict()
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(history,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            metric_aic_dict.update({(param,param_seasonal).results.aic})
        except:
            continue

In [ ]:
#ARIMA(1, 0, 2)x(0, 0, 2, 12)12 - AIC:181.7547617641269
#ARIMA(0, 1, 0)x(2, 2, 0, 12)12 - AIC:198.82449368899
#ARIMA(0, 1, 1)x(2, 2, 0, 12)12 - AIC:195.2215235269675
#ARIMA(0, 2, 2)x(2, 2, 0, 12)12 - AIC:186.52096621003744
#ARIMA(1, 1, 0)x(2, 2, 1, 12)12 - AIC:187.1520618303596
#ARIMA(1, 1, 1)x(2, 2, 0, 12)12 - AIC:186.77546901132936
#ARIMA(1, 1, 1)x(2, 2, 1, 12)12 - AIC:188.7005403352341
#ARIMA(1, 2, 1)x(2, 2, 0, 12)12 - AIC:183.0945070804955
#ARIMA(1, 2, 2)x(2, 2, 0, 12)12 - AIC:177.92491460305126
#ARIMA(2, 0, 0)x(2, 2, 0, 12)12 - AIC:186.1857038455962
#ARIMA(2, 0, 2)x(2, 2, 0, 12)12 - AIC:182.78979559835588
#ARIMA(2, 1, 0)x(2, 2, 0, 12)12 - AIC:179.79328912737333
#ARIMA(2, 1, 0)x(2, 2, 1, 12)12 - AIC:181.14363816582275
#ARIMA(2, 1, 2)x(2, 2, 1, 12)12 - AIC:178.13615684891985
#ARIMA(2, 2, 1)x(2, 2, 0, 12)12 - AIC:172.58196558190537#
#ARIMA(2, 2, 2)x(2, 2, 0, 12)12 - AIC:172.60657003313923


In [ ]:
mod=sm.tsa.statespace.SARIMAX(history, order=(2,0,0), seasonal_order=(2,0,0,12),enforce_stationarity=False,
                                            enforce_invertibility=False)
results = mod.fit()
results.summary()

MODEL DIAGNOSIS PLOT TO CHECK RESIDUAL

In [ ]:
results.plot_diagnostics(figsize=(16,8))
plt.show()

In [ ]:
# plot forecasts against actual outcomes
plt.plot(test)
plt.plot(predictions, color='red')
plt.plot(test,label="Original test TB incidence")
plt.plot(predictions,label="Predicted TB incidence")
plt.legend( loc='upper left')
plt.show()

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
rmse =sqrt(mean_squared_error(test, predictions))
print(rmse)

In [ ]:
# walk-forward validation
for t in range(len(test)):
	model = sm.tsa.statespace.SARIMAX(history, order=(1,0,2), seasonal_order=(2,0,2,12),enforce_stationarity=False,
                                            enforce_invertibility=False)
	model_fit = model.fit() 
  
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

model_fit.summary()

In [ ]:
#pf['forecast'] = results.predict(start = 102, end= 114, dynamic= True)  
#pf[['tbcases', 'forecast']].plot(figsize=(12, 8))

In [ ]:
#ARIMA(0, 1, 0)x(2, 2, 0, 12)12 - AIC:198.82449368899
#ARIMA(0, 1, 1)x(2, 2, 0, 12)12 - AIC:195.2215235269675
#ARIMA(0, 1, 1)x(2, 2, 1, 12)12 - AIC:197.21601675994546
#ARIMA(0, 1, 2)x(2, 2, 0, 12)12 - AIC:196.10610698970928
#ARIMA(0, 1, 2)x(2, 2, 1, 12)12 - AIC:198.07856722512545
#ARIMA(0, 2, 1)x(2, 2, 0, 12)12 - AIC:192.73520175511138
#ARIMA(0, 2, 2)x(2, 2, 0, 12)12 - AIC:186.52096621003744
#ARIMA(1, 0, 0)x(2, 2, 0, 12)12 - AIC:196.9384420489906
#ARIMA(1, 0, 0)x(2, 2, 1, 12)12 - AIC:198.90447550441286
#ARIMA(1, 0, 1)x(2, 2, 0, 12)12 - AIC:196.9216669407432
#ARIMA(1, 0, 1)x(2, 2, 1, 12)12 - AIC:198.92166397562914
#ARIMA(1, 0, 2)x(2, 2, 0, 12)12 - AIC:196.42104366328348
#ARIMA(1, 0, 2)x(2, 2, 1, 12)12 - AIC:189.37155654191872
#ARIMA(1, 1, 0)x(2, 2, 0, 12)12 - AIC:186.05293168247476
#ARIMA(1, 1, 0)x(2, 2, 1, 12)12 - AIC:187.1520618303596
#ARIMA(1, 1, 1)x(2, 2, 0, 12)12 - AIC:186.77546901132936
#ARIMA(1, 1, 1)x(2, 2, 1, 12)12 - AIC:188.7005403352341
#ARIMA(1, 1, 2)x(2, 2, 1, 12)12 - AIC:188.9934080871116
#ARIMA(1, 2, 0)x(2, 2, 0, 12)12 - AIC:192.14247217010475
#ARIMA(1, 2, 2)x(2, 2, 0, 12)12 - AIC:177.92491460305126
#ARIMA(2, 0, 0)x(2, 2, 0, 12)12 - AIC:186.1857038455962
#ARIMA(2, 0, 0)x(2, 2, 1, 12)12 - AIC:187.23555192402318
#ARIMA(2, 0, 1)x(2, 2, 0, 12)12 - AIC:187.3567221543439
#ARIMA(2, 1, 0)x(2, 2, 0, 12)12 - AIC:179.79328912737333
#ARIMA(2, 1, 0)x(2, 2, 1, 12)12 - AIC:181.14363816582275
#ARIMA(2, 1, 1)x(2, 2, 1, 12)12 - AIC:183.12865722937335
#ARIMA(2, 1, 2)x(2, 2, 0, 12)12 - AIC:176.3816755432984
#ARIMA(2, 1, 2)x(2, 2, 1, 12)12 - AIC:178.13615684891985
#ARIMA(2, 2, 0)x(2, 2, 0, 12)12 - AIC:177.35891104434828
#ARIMA(2, 2, 1)x(2, 2, 0, 12)12 - AIC:172.58196558190537
#ARIMA(2, 2, 2)x(2, 2, 0, 12)12 - AIC:172.60657003313923